# 연습문제3 개요

1. 충북 관광 데이터에 대하여 조별로 사전기반의 감성분석을 수행하시오
- 법주사.xlsx, 충주호.xlsx, 청남대.xlsx, 화양구곡.xlsx, 산막이옛길.xlsx 중 1개 데이터를 선택하여 진행
- 자료를읽고,어떤 분석을 할지 분석계획서 수립
- 분석계획서작성 – 분석 – 조별결과발표(분석내용 및 방법)
- 결측값이 다수 있으니 적절히 처리
- 홍보성 글과 뉴스 등 관련없는 글은 데이터내용을 활용하여 가능한 제외처리(규칙기반)
    - <u>**홍보성글과 뉴스를 버리지 말고 잘 가지고 있어. 나중에 제외사전으로 사용할 예정.**</u>
- 앞에서 만든 감정사전과 pytest 폴더에 있는 positive.txt, negative.txt 파일을 이용하여 사전을 보완
- 본 프로젝트를 진행하며 자유롭게 감정 단어를 추가

2. 한눈에 보는 전체 프로세스
- 자료 선택 및 분석계획서 작성
- 자료 탐색
- 전처리(결측값 제거)
- 제외처리(홍보성 글들)
- 빈도분석
- (사전추가)
- 감성분석
- 보고용 그래프 그리기

## 화양구곡 관련 블로그 댓글 말뭉치 가져오기

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)

path = '/Users/jsha/gjai/nlp/pytest/'
file='화양구곡.xlsx'

hg_df = pd.read_excel(path+file)
hg_df.shape

(1939, 7)

## 전처리 하기

### 결측치 제거

In [2]:
# 내용에서의 결측치 제거
hg_df.isnull().sum()
hg_df.dropna(subset=['내용'], inplace=True)
print(len(hg_df))

1882


### 화양이라는 단어가 포함되지 않은 "내용"은 제거

In [3]:
def ex_hg(string):
    ll = ['화양계곡', '화양구곡']
    for l in ll:    
        if l in string:
            return True
        
dummy = hg_df['내용'].apply(ex_hg)
dummy[dummy.isnull()] = False

hg_df = hg_df[dummy]
len(hg_df)

1392

### 홍보성 글 or 뉴스 글 제거
- 이를 위해 먼저, 홍보성 글이나 뉴스 글을 대변하는 단어를 찾아야 하는데, 이를 hg_df의 '제목'이나 '내용'을 훝으면서 대략적으로 뽑아냄.

In [4]:
# df[['글제목', '내용', '공감수', '댓글수']][1800:]

In [5]:
# 제외단어 = """
# 특별전
# 투어
# 사례
# 아르바이트
# 알바
# 중국
# 일본
# 출판
# 여행사
# 주택
# 매매
# 토지
# 부동산
# 변기
# 뉴스
# 영단어
# 수익
# 렌트카
# 지리산
# 신축
# 기자
# 일보
# 부지
# 정투
# 방송
# 클럽
# 캠페인
# 안내
# 모집
# 캠프
# 손세차
# 료칸
# 인터뷰
# 대회
# 임대
# Q&A
# 공지
# 광고
# 병원
# 미디어
# 패키지
# 행사
# 총회
# 회의
# 상담
# 경북
# 강원
# 경남
# 전남
# 전북
# 대전
# 건대
# 서울
# 대표전화
# """

In [6]:
# excl_word_list = 제외단어.split('\n')
# excl_word_list.pop()
# excl_word_list.remove('')
# excl_word_list
# string = '\t'.join(excl_word_list)
# with open(path+'excl_word_jsha.txt', 'w') as f:
#     f.write(string)
    
with open(path+'excl_word_jsha.txt', 'r') as f:
    data = f.read()
excl_word_list = data.split('\t')

# print(excl_word_list)

def ex_news_hg(string):
    for l in excl_word_list:    
        if l in string:
            return False
        
dummy = hg_df['글제목'].apply(ex_news_hg)
dummy[dummy.isnull()] = True
dummy.value_counts()

new_df = hg_df[dummy].copy()


## 빈도 분석

### 형태소 분석

In [9]:
from tqdm import tqdm # 이거 엄청 느리네. 쓰지말자.
import rhinoMorph

rn = rhinoMorph.startRhino()


stopwords_ko = ["하다", "있다", "되다", "그", "않다", "없다", "나", "말", 
                "사람", "이", "보다", " 한", "때", "년", "같다", "대하다", 
                "일", "이", "생각", "위하다", "때문", "그것", "그러나", "가다",
                " 받다", "그렇다", "알다", "사회", "더", "그녀", "문제", 
                "오다", "그리고", "크다", "속"]

def morph_list(string):
    morphed_list = rhinoMorph.onlyMorph_list(rn, string,
                    pos= ['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 
                          'MM', 'MAG', 'MAJ'], eomi=True)
    morphed_list = [text for text in morphed_list if text 
                         not in stopwords_ko]
    return morphed_list

new_df['morph_list'] = new_df['내용'].apply(morph_list)

filepath:  /Users/jsha/opt/anaconda3/envs/nlp3710/lib/python3.7/site-packages
classpath:  /Users/jsha/opt/anaconda3/envs/nlp3710/lib/python3.7/site-packages/rhinoMorph/lib/rhino.jar
RHINO started!


,id,글제목,글쓴이,내용,태그,공감수,댓글수,morph_list
0,1,화양동계곡 화양구곡 산책 길에서,베리이모,새해 아침성당 가서 미사 드리고오는 길에 아들이랑산책왔어요.화양동계곡혹은 화양구곡이...,,16.0,25.0,"[새해, 아침, 성당, 미사, 드리다, 길, 아들, 산책, 화양동계곡, 혹은, 화양..."
2,3,괴산 화양계곡 속리산국립공원 힐링 최고^^,Leaping Rooster,괴산 화양계곡 속리산국립공원 힐링 최고^^안녕하세요! 새해 첫 주말 괴산 화양계곡에...,,11.0,14.0,"[괴산, 화양계곡, 속리산국립공원, 힐링, 최고, ^^, 안녕, 새해, 첫, 주말,..."
6,7,상주귀농 실천하다!,우복동온달,수 년간 고민하고 지난 1년간 준비해서 드뎌서울 마포구 상암동에서 상주시 화북면 청...,#귀농귀촌 #문장대 #쌍용계곡 #우복동가든황토펜션 #농협대 #상주귀농공동체 #경기...,3.0,4.0,"[수, 간, 고민하, 지나다, 준비, 마포구, 상암동, 상주시, 화북, 청화, 번지..."
7,8,"[괴산] 가령산, 도명산",사네드레,"송년 산행이라 특별하고, 신년 산행이라 남다를것도 없습니다.서해로 넘어가는 해를 바...",#ILCE-7 #ILCE-6000 #가령산 #도명산,18.0,18.0,"[송년, 산행, 특별, 신년, 산행, 남다, 다르다, 서해, 로, 넘다, 해, 바라..."
10,11,괴산 화양계곡과 도명산,옥시기,○ 2016.1. 5 (화) 미지회원 6명○ 화양동탐방센터(08:40) ~ 하양계곡...,NaN,7.0,3.0,"[화, 미지, 회원, 화양동, 탐방, 센터, 하양, 계곡, 첨성대, 도명산, 낙영산..."


In [13]:
morphs = new_df['morph_list'].sum()

In [14]:
from collections import Counter

counter = Counter(morphs)
counter

Counter({'새해': 7,
         '아침': 355,
         '성당': 21,
         '미사': 6,
         '드리다': 122,
         '길': 1526,
         '아들': 103,
         '산책': 160,
         '화양동계곡': 97,
         '혹은': 33,
         '화양구곡': 1457,
         '아름답다': 620,
         '곳': 2549,
         '주차장': 563,
         '전': 600,
         '곡': 1741,
         '경천': 280,
         '벽': 328,
         '지나치다': 86,
         '쉽다': 156,
         '화양계곡': 1846,
         '펜션': 2123,
         '편의점': 25,
         '식당': 354,
         '어서': 96,
         '예약': 299,
         '차': 535,
         '가지다': 246,
         '들다': 863,
         '수도': 38,
         '가족': 676,
         '걷다': 449,
         '들리다': 80,
         '좋다': 2751,
         '산행': 952,
         '좋아하다': 244,
         '계곡': 2927,
         '안': 808,
         '도명산': 902,
         '구름': 242,
         '그림자': 110,
         '맑다': 661,
         '비치다': 134,
         '운영': 459,
         '지나다': 666,
         '우암': 397,
         '송시열': 677,
         '사당': 51,
         '좀': 454,
         '